In [ ]:
# Cell 0: 라이브러리 임포트 및 시드 고정, 환경 설정

#  기본 및 파일 시스템 관련 (Basic & File System)
import os
import json # JSON 데이터 처리
import csv  # CSV 파일 처리
import time # 시간 관련 기능
import joblib # 스케일러, 모델 등 객체 저장 및 로드
import gc # 가비지 컬렉션 추가

# 데이터 처리 및 분석 (Data Handling & Analysis)
import numpy as np
import pandas as pd
from openai import OpenAI # OpenAI API (GPT 모델 등) - 현재 코드에서는 직접 사용되지 않으나, 기능상 분류

# 금융 데이터 수집 (Financial Data Acquisition)
import yfinance as yf

# 자연어 처리 및 감성 분석 (Natural Language Processing / Sentiment Analysis)
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer # VADER 감성 분석
from textblob import TextBlob # TextBlob 감성 분석 및 NLP
from transformers import pipeline # Hugging Face Transformers 라이브러리 (다양한 NLP 작업)

# 머신러닝 및 딥러닝 (Machine Learning & Deep Learning)
# from openai import OpenAI # OpenAI API (GPT 모델 등) - 현재 코드에서는 직접 사용되지 않으므로 주석 처리 가능
import tensorflow as tf
from tensorflow.keras.models import load_model, Sequential # 모델 로드 및 구성
from tensorflow.keras.optimizers import Adam # 옵티마이저
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization # 레이어 종류
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint # 콜백 함수들
from sklearn.preprocessing import StandardScaler, MinMaxScaler # MinMaxScaler 추가 (Cell 7에서 사용)

# 웹 스크레이핑 및 자동화 (Web Scraping & Automation)
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

# 유틸리티 (Utilities)
from tqdm import tqdm # 진행 상황 표시
from datetime import datetime, timedelta # 날짜 및 시간 처리
import concurrent.futures # 병렬 처리 (현재 코드에서는 직접 사용되지 않으나, 기능상 분류)

# 오늘 날짜
today = datetime.today().strftime('%Y-%m-%d')

# 랜덤 시드 고정
np.random.seed(42)
tf.random.set_seed(42)

# ─── 환경 설정 ─────────────────────────────────────────────────────────
# OPENAI_API_KEY는 환경 변수 등에서 로드하는 것을 권장합니다.
OPENAI_API_KEY = ""
MODEL            = "gpt-4o-mini"
PAGE_LOAD_TIMEOUT = 30         # 드라이버 페이지 로드 최대 대기 (초)
OPENAI_TIMEOUT     = 60        # OpenAI API 최대 대기 (초)
MAX_RETRIES        = 2         # 페이지 로드 재시도 횟수
MAX_WORKERS        = 8         # ThreadPoolExecutor 워커 수


pred_date_str  = '2025-06-06'  # 예측하는 날짜
proc_file      = 'data.csv'  # 데이터가 저장된 파일 (이후 main_df로 관리)

# model_dir이 정의된 후에 mapping_path 정의
model_dir      = 'models'
if not os.path.exists(model_dir):
    os.makedirs(model_dir)

sent_path      = './ticker_agg_sentiment.csv'  # 감성 파일 이름 (사용 여부 일관성 필요)
output_file    = 'submission.csv'  # 제출 파일이 생성되는 이름
train_date_str = '2025-06-04' # 학습 데이터의 마지막 날짜 기준
prediction_horizon = 2 # 예측하려는 기간 (n일 뒤)
min_data_for_training = 20 # 학습에 필요한 최소 샘플 수 (타겟 생성 후 기준)

sequence_length = 20 # 예: 과거 20일 데이터 사용

# 1) 뉴스 로드
input_crawled_articles = './crawled_articles.csv'  # 크롤링된 뉴스들 저장하는 곳
output_sentiment_file = './ticker_agg_sentiment.csv' # 셀0의 sent_path와 동일하게

# IGNORE THIS
mapping_path   = os.path.join(model_dir, 'feature_cols_map.json')

# 전역 DataFrame 변수 (메모리 최적화를 위해 이후 셀에서 재사용)
main_df = pd.DataFrame()
sentiment_df = pd.DataFrame()

# 예측하는 종목들
tickers_to_scrape = [
        "005930:KRX", "000660:KRX", "207940:KRX", "373220:KRX", "005380:KRX",
        "012450:KRX", "005935:KRX", "105560:KRX", "329180:KRX", "068270:KRX",
        "000270:KRX", "035420:KRX", "055550:KRX", "042660:KRX", "012330:KRX",
        "138040:KRX", "028260:KRX", "005490:KRX", "009540:KRX", "259960:KRX",
        "086790:KRX", "000810:KRX", "034020:KRX", "032830:KRX", "035720:KRX",
        "015760:KRX", "010130:KRX", "011200:KRX", "051910:KRX", "033780:KRX"
]

In [ ]:
# Cell 1: 뉴스 링크 저장 (스레드 사용 개선)
def init_driver_cell1(headless=True):
    opts = Options()
    if headless:
        opts.add_argument("--headless")
    opts.add_argument("--no-sandbox")
    opts.add_argument(f"user-agent=Mozilla/5.0 (compatible; ThreadBot/{time.time()})")
    return webdriver.Chrome(options=opts)

def scrape_stock_news_for_ticker(ticker_info):
    ticker = ticker_info['ticker']
    lang = ticker_info.get('lang', "english")

    driver = init_driver_cell1(headless=True)
    news_for_this_ticker = []
    try:
        url = f"https://www.google.com/finance/quote/{ticker}?hl={lang}"
        driver.get(url)

        wait = WebDriverWait(driver, 10)
        wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, "div.YMlKec.fxKbKc, div.zzDege")))

        news_container_candidates = ["div.yY3Lee", "div.AoCdqe"] # 뉴스 아이템을 포함할 가능성이 있는 컨테이너 선택자
        news_loaded_condition = EC.presence_of_element_located(
            (By.CSS_SELECTOR, ", ".join(news_container_candidates))
        )
        wait.until(news_loaded_condition) # 뉴스 컨테이너 중 하나라도 로드될 때까지 대기

        found_elements = []
        for selector in news_container_candidates:
            elements = driver.find_elements(By.CSS_SELECTOR, selector)
            if elements:
                found_elements.extend(elements)

        temp_urls = set() # URL 중복 제거를 위해 set 사용
        for block in found_elements:
            try:
                link_elem = None
                # 가능한 링크 선택자들을 순회하며 href 속성을 가진 첫 번째 요소를 찾습니다.
                possible_link_selectors = ["div.z4rs2b > a", "a.TxRU9d", "a"]
                for link_selector in possible_link_selectors:
                    try:
                        link_elem = block.find_element(By.CSS_SELECTOR, link_selector)
                        if link_elem and link_elem.get_attribute("href"):
                            break # 유효한 링크를 찾으면 중단
                    except: # find_element 실패 시
                        link_elem = None
                
                if link_elem:
                    href = link_elem.get_attribute("href")
                    # 유효한 외부 뉴스 링크인지 확인 (구글 금융 내부 링크 제외)
                    if href and href.startswith("http") and "google.com/finance" not in href:
                        temp_urls.add(href)
            except Exception: # 개별 뉴스 블록 처리 중 오류 발생 시 다음 블록으로 넘어감
                continue
        
        news_for_this_ticker = [{'ticker': ticker, 'url': url_str} for url_str in temp_urls]
        # print(f"[{ticker}] found {len(news_for_this_ticker)} unique news links")

    except Exception as e:
        print(f"[{ticker}] 스크래핑 오류: {e}")
    finally:
        driver.quit()
    
    # 함수 반환 시 (ticker, url) 튜플 리스트로 변경
    return [(item['ticker'], item['url']) for item in news_for_this_ticker]

# 이 스크립트를 직접 실행할 때만 아래 코드가 실행되도록 __main__ 가드 사용
if __name__ == "__main__":    
    tasks = [{'ticker': t} for t in tickers_to_scrape]
    all_news_results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        future_to_ticker = {executor.submit(scrape_stock_news_for_ticker, task): task['ticker'] for task in tasks}
        
        for future in tqdm(concurrent.futures.as_completed(future_to_ticker), total=len(tasks), desc="뉴스 링크 스크래핑"):
            ticker_name = future_to_ticker[future]
            try:
                news_items = future.result() # [(ticker, url), ...] 형태
                if news_items:
                    all_news_results.extend(news_items) # news_items는 이미 리스트이므로 extend 사용
            except Exception as exc:
                print(f"티커 {ticker_name} 처리 중 예외 발생: {exc}")

    output_filename = "news_urls.csv"
    with open(output_filename, "w", newline="", encoding="utf-8-sig") as f:
        writer = csv.writer(f)
        writer.writerow(["ticker", "url"])
        writer.writerows(all_news_results) # all_news_results는 (ticker, url) 튜플의 리스트

    print(f"완료! 총 {len(all_news_results)}개의 링크가 {output_filename}에 저장되었습니다.")
# else:
#     print("Cell 1: 뉴스 링크 저장 코드가 로드되었습니다. (실행은 __main__ 블록 내에 있음)")

In [ ]:
# Cell 2: 저장된 뉴스 링크들 기반으로 AI활용해서 동적 크롤링

def init_dynamic_scraping_driver(headless=True): # 함수명 변경 (init_driver_cell6 -> init_dynamic_scraping_driver)
    opts = Options()
    if headless:
        opts.add_argument("--headless")
        opts.add_argument("--disable-gpu") # headless 모드에서 GPU 비활성화는 일반적
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage") # Docker 또는 CI 환경에서 메모리 문제 방지
    opts.add_argument("--window-size=1920x1080") # 페이지 레이아웃에 영향 줄 수 있음
    # User-agent 문자열에 타임스탬프를 넣어 매번 다른 값으로 보이도록 하는 것은 좋으나, 과도하게 자주 변경되면 오히려 이상행위로 감지될 수 있음
    user_agent = f"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.{int(time.time())%4000 + 4000}.{int(time.time())%100 + 100} Safari/537.36"
    opts.add_argument(f"user-agent={user_agent}")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"]) # 자동화 탐지 회피
    opts.add_experimental_option('useAutomationExtension', False) # 자동화 확장 기능 사용 안 함
    opts.add_argument('--disable-blink-features=AutomationControlled') # 자동화 제어 기능 비활성화
    driver = webdriver.Chrome(options=opts)
    driver.set_page_load_timeout(PAGE_LOAD_TIMEOUT) # Cell 0의 PAGE_LOAD_TIMEOUT 사용
    return driver

def extract_title_and_body_with_ai(full_text: str) -> dict:
    if 'client' not in globals() or globals()['client'] is None: # globals()를 통해 명시적으로 전역 변수 확인
        global client # client 사용 전에 global 선언
        print("Warning: OpenAI client re-initialized in extract_title_and_body_with_ai. Should be initialized in Cell 0.")
        client = OpenAI(api_key=OPENAI_API_KEY) # OPENAI_API_KEY는 Cell 0에서 정의되어야 함

    system_prompt = (
        "당신은 뉴스 크롤러입니다. 입력된 전체 텍스트에서 '제목'과 '본문'만을 "
        "추출하여 정확히 JSON 형식으로 응답해주세요. "
        "JSON 키는 'title'과 'body' 이어야 합니다."
    )
    user_prompt = f"전체 페이지 텍스트:\\n\\n```{full_text}```\\n\\n위에서 제목과 본문을 구분하여 JSON으로만 응답해 주세요."
    
    try:
        resp = client.chat.completions.create(
            model=MODEL, # Cell 0의 MODEL 사용
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_prompt}
            ],
            temperature=0, # 일관된 결과를 위해 temperature 0
            timeout=OPENAI_TIMEOUT # Cell 0의 OPENAI_TIMEOUT 사용
        )
        content = resp.choices[0].message.content.strip()
        
        # AI 응답에서 JSON 코드 블록 마커(```json ... ``` 또는 ``` ... ```) 제거
        if content.startswith("```"):
            # "```json"으로 시작하면 "```json" 제거, 아니면 "```" 제거
            content = content[len("```json"):] if content.startswith("```json") else content[len("```"):]
            if content.endswith("```"):
                 content = content[:-len("```")] # 끝의 "```" 제거
            content = content.strip()
        return json.loads(content) # JSON 파싱
    except Exception as e:
        print(f"AI 응답 처리 또는 API 호출 오류: {e}")
        return {"title": "", "body": ""} # 오류 시 빈 값 반환


def process_row(row: dict) -> dict:
    ticker = row["ticker"]
    url    = row["url"]
    driver = init_dynamic_scraping_driver(headless=True)
    wait   = WebDriverWait(driver, 10) # 내부 요소 대기 시간 (페이지 로드 타임아웃과 별개)
    result = {"ticker": ticker, "url": url, "title": "", "body": ""}

    try:
        for attempt in range(1, MAX_RETRIES + 2): # 1부터 시작 (MAX_RETRIES=0 이면 1번 시도)
            try:
                driver.get(url)
                # body 태그가 존재할 때까지 기다리는 것으로 페이지 기본 로드 확인
                wait.until(EC.presence_of_element_located((By.TAG_NAME, "body")))
                break # 성공 시 루프 탈출
            except TimeoutException:
                print(f"[{ticker}] ⚠️ 페이지 로드 타임아웃({attempt}/{MAX_RETRIES+1}): {url}")
                if attempt > MAX_RETRIES:
                    print(f"[{ticker}] ❌ 스킵 (최대 재시도 초과): {url}")
                    return result # 최대 재시도 초과 시 현재 result 반환
            except Exception as e: # TimeoutException 외 다른 페이지 로드 에러
                print(f"[{ticker}] ❌ 페이지 로드 중 에러: {e}, URL: {url}")
                return result # 이 경우 재시도 없이 바로 반환

        # 페이지 로드 성공 후 본문 텍스트 추출
        try:
            full_text = driver.find_element(By.TAG_NAME, "body").text
        except Exception as e:
            print(f"[{ticker}] ❌ 본문 텍스트 추출 실패: {e}, URL: {url}")
            return result # 본문 추출 실패 시 현재 result 반환

        # AI를 사용하여 제목과 본문 추출
        try:
            data = extract_title_and_body_with_ai(full_text)
            result["title"] = data.get("title", "").strip()
            result["body"]  = data.get("body", "").strip().replace("\n", "\\n") # 개행문자 이스케이프 처리
            # print(f"[{ticker}] ✔️ AI 추출 성공: {url}") # 성공 로그는 너무 많을 수 있으므로 주석 처리
        except json.JSONDecodeError as e_json: # AI 응답이 유효한 JSON이 아닌 경우
            print(f"[{ticker}] ❌ AI 응답 JSON 파싱 실패: {e_json}, URL: {url}")
        except Exception as e_ai: # OpenAI API 호출 실패 등 기타 AI 관련 에러
            print(f"[{ticker}] ❌ AI 추출 중 에러: {e_ai}, URL: {url}")
            # AI 추출 실패 시에도 result에는 ticker, url이 이미 있고 title, body는 초기값 "" 유지

    finally: # 어떤 경우에도 드라이버는 종료되도록 보장
        driver.quit()
        
    return result


def crawl_articles(input_csv="news_urls.csv", output_csv="crawled_articles.csv"):
    if not os.path.exists(input_csv):
        print(f"오류: 입력 파일 '{input_csv}'을(를) 찾을 수 없습니다.")
        return

    try:
        with open(input_csv, newline="", encoding="utf-8-sig") as fin:
            reader = list(csv.DictReader(fin))
    except Exception as e:
        print(f"오류: 입력 파일 '{input_csv}' 읽기 실패: {e}")
        return
        
    if not reader:
        print(f"정보: 입력 파일 '{input_csv}'에 데이터가 없습니다.")
        # 빈 출력 파일 생성 (헤더만)
        with open(output_csv, "w", newline="", encoding="utf-8-sig") as fout:
            fieldnames = ["ticker", "url", "title", "body"]
            writer = csv.DictWriter(fout, fieldnames=fieldnames)
            writer.writeheader()
        return

    with open(output_csv, "w", newline="", encoding="utf-8-sig") as fout:
        fieldnames = ["ticker", "url", "title", "body"]
        writer = csv.DictWriter(fout, fieldnames=fieldnames)
        writer.writeheader()

        with concurrent.futures.ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
            # 각 행(row_data)에 대해 process_row 함수를 병렬로 실행
            futures = [executor.submit(process_row, row_data) for row_data in reader]
            
            # 작업 완료되는 순서대로 결과 처리 (tqdm으로 진행 상황 표시)
            for future in tqdm(concurrent.futures.as_completed(futures), total=len(reader), desc="뉴스 본문 크롤링 및 AI 추출"):
                try:
                    res = future.result() # process_row의 반환값 (dict)
                    writer.writerow(res)
                except Exception as e_future:
                    # process_row 내부에서 예외가 발생했으나 처리되지 않은 경우 (이론상 process_row는 항상 dict를 반환)
                    # 또는 future.result() 자체에서 다른 문제가 발생한 경우
                    print(f"행 처리 중 예상치 못한 오류 (future 결과 처리): {e_future}")
                    # 이 경우 해당 행은 CSV에 기록되지 않을 수 있음. 오류 로깅 및 기본값 기록 등 고려.

    print(f"\n✅ 크롤링 및 AI 추출 완료! `{output_csv}` 에 저장되었습니다.")

if __name__ == "__main__":
    # Cell 0에서 OpenAI client 초기화
    client = OpenAI(api_key=OPENAI_API_KEY)
    
    print(f"Using OPENAI_API_KEY: {OPENAI_API_KEY[:10]}...")
    print(f"Using MODEL: {MODEL}")
    print(f"Using PAGE_LOAD_TIMEOUT: {PAGE_LOAD_TIMEOUT}")
    print(f"Using OPENAI_TIMEOUT: {OPENAI_TIMEOUT}")
    print(f"Using MAX_RETRIES: {MAX_RETRIES}")
    print(f"Using MAX_WORKERS for crawling: {MAX_WORKERS}")

    # input_crawled_articles는 Cell 0에서 정의된 변수 (output_csv와 같아야 함)
    crawl_articles(input_csv="news_urls.csv", output_csv=input_crawled_articles)
# else:
#     print("Cell 2: AI 기반 크롤링 코드가 로드되었습니다. (실행은 __main__ 블록 내에 있음)")

In [ ]:
# Cell 3: 고급화된 티커별 감성 지표 계산·저장 (Optimized Sentiment Calculation)
import warnings

try:
    from tqdm import TqdmWarning
    warnings.filterwarnings("ignore", category=TqdmWarning) # tqdm 관련 경고 무시
except ImportError: # tqdm.tqdm_notebook.TqdmSynchronisationWarning 같은 경우
    warnings.filterwarnings("ignore", message=".*IProgress not found.*") # IProgress 관련 경고 무시


# Define expected columns for an empty output file
expected_sentiment_cols = [
    'ticker', 'compound_mean', 'compound_std', 'compound_sum', 'compound_median', # median 추가됨
    'pos_mean', 'pos_sum', 'neg_mean', 'neg_sum', 'neu_mean', 'neu_sum',
    'tb_polarity_mean', 'tb_polarity_std', 'tb_polarity_median', # median 추가됨
    'tb_subjectivity_mean', 'tb_subjectivity_std', 'tb_subjectivity_median', # median 추가됨
    'bert_star_mean', 'bert_star_std', 'bert_star_median', # median 추가됨
    'bert_score_normalized_mean', 'bert_score_normalized_std', 'bert_score_normalized_median', # median 추가됨
    'sentiment_score_custom_mean', 'sentiment_score_custom_std', 'sentiment_score_custom_median', # median 추가됨
    'strong_pos_count', 'strong_neg_count', 'total_articles',
    'pos_neg_ratio', 'sentiment_intensity_ratio'
]

# input_crawled_articles, output_sentiment_file는 Cell 0에서 정의됨
if not os.path.exists(input_crawled_articles):
    print(f"오류: 감성 분석을 위한 입력 파일 '{input_crawled_articles}'이 없습니다. 이전 단계를 먼저 실행하세요.")
    # 빈 DataFrame을 만들고 기대 컬럼으로 채워서 저장
    df_final_empty = pd.DataFrame(columns=expected_sentiment_cols)
    df_final_empty.to_csv(output_sentiment_file, index=False)
    print(f"빈 감성 지표 파일 '{output_sentiment_file}'이(가) 생성되었습니다 (기대 컬럼 포함).")
    df_news = pd.DataFrame(columns=['ticker', 'title', 'body']) # 이후 코드 실행을 위해 빈 df_news 생성
else:
    df_news = pd.read_csv(input_crawled_articles)

# title과 body가 NaN인 경우 빈 문자열로 채우고, 두 컬럼을 합쳐 'text' 컬럼 생성
# 메모리 효율성을 위해 str.cat 사용 가능: df_news['text'] = df_news['title'].fillna('').str.cat(df_news['body'].fillna(''), sep=' ')
df_news['title'] = df_news['title'].fillna('')
df_news['body'] = df_news['body'].fillna('')
df_news['text'] = df_news['title'] + ' ' + df_news['body']
df_news['text'] = df_news['text'].str.strip().replace('', np.nan) # 공백만 있거나 빈 문자열을 NaN으로
df_news.dropna(subset=['text'], inplace=True) # 'text'가 NaN인 행 제거 (분석할 내용이 없는 경우)

if df_news.empty:
    print("감성 분석을 진행할 뉴스가 없습니다 (파일이 비어있거나 모든 텍스트가 비어있음).\n")
    df_final_empty = pd.DataFrame(columns=expected_sentiment_cols)
    if 'ticker' not in df_final_empty.columns and 'ticker' in expected_sentiment_cols: # 'ticker' 컬럼 보장
         df_final_empty = pd.DataFrame(columns = ['ticker'] + [col for col in expected_sentiment_cols if col != 'ticker'])
    df_final_empty.to_csv(output_sentiment_file, index=False)
    print(f"빈 감성 지표 파일 '{output_sentiment_file}'이(가) 생성되었습니다 (기대 컬럼 포함).")
else:
    # 2) VADER 분석
    analyzer = SentimentIntensityAnalyzer()
    # NaN이나 빈 문자열에 대해 안전하게 처리 (str(x)로 변환)
    vader_scores_series = df_news['text'].apply(lambda x: analyzer.polarity_scores(str(x)) if pd.notnull(x) else {})
    df_v = pd.DataFrame(vader_scores_series.tolist(), index=df_news.index)

    # 3) TextBlob 분석
    def safe_textblob_sentiment(text):
        try:
            # NaN, None, 또는 공백만 있는 문자열 처리
            if pd.isnull(text) or not str(text).strip():
                 return TextBlob("").sentiment # 빈 TextBlob 객체의 감성 (보통 (0,0))
            return TextBlob(str(text)).sentiment
        except Exception: # 예상치 못한 오류 발생 시
            return TextBlob("").sentiment # 기본값 반환
    tb_scores_series = df_news['text'].apply(safe_textblob_sentiment)
    df_tb = pd.DataFrame(tb_scores_series.tolist(), columns=['tb_polarity','tb_subjectivity'], index=df_news.index)

    # 4) BERT 계열 감성 분석
    df_bert = pd.DataFrame(index=df_news.index)
    # BERT 분석 실패 또는 실행 안 될 경우를 대비한 기본값
    df_bert['bert_star'] = 3 # 중립 (1~5점 척도에서 중간)
    df_bert['bert_score_normalized'] = 0.0 # 중립 (-1 ~ 1 범위에서 0)

    try:
        device_id = 0 if tf.config.list_physical_devices('GPU') else -1 # GPU 사용 가능하면 0, 아니면 -1(CPU)
        if device_id == 0: print("BERT 감성 분석에 GPU를 사용합니다.")
        else: print("BERT 감성 분석에 CPU를 사용합니다. (GPU를 찾을 수 없음)")
        
        # 금융 특화 모델 사용 시 라이선스, 성능, 호환성 확인 필요
        # sentiment_pipe = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=device_id) 
        sentiment_pipe = pipeline(
            "sentiment-analysis",
            model="nlptown/bert-base-multilingual-uncased-sentiment", # 다국어 지원 모델
            device=device_id,
            # truncation=True # 파이프라인에서 직접 truncation 지원하는지 확인. 명시적 토큰화가 더 안전할 수 있음.
        )
        
        def get_bert_scores(text):
            try:
                text_to_analyze = str(text)
                # NaN, None, 또는 공백만 있는 문자열이면 기본값 반환
                if not text_to_analyze.strip() or pd.isnull(text_to_analyze):
                    return 3, 0.0 
                
                r = sentiment_pipe(text_to_analyze[:512])[0] # BERT 모델 최대 입력 길이 고려
                
                star = int(r['label'].split()[0]) # 예: "5 stars" -> 5
                normalized_score = (star - 3) / 2 # 1~5점 척도를 -1 ~ 1 범위로 정규화 (3점이 0)
                return star, normalized_score
            except Exception as e:
                # print(f"BERT 분석 오류: {e} for text: {str(text)[:50]}...") # 디버깅 시 사용
                return 3, 0.0 # 오류 발생 시 기본값 반환
        
        # tqdm을 사용하여 진행 상황 표시 (데이터가 클 경우 유용)
        # bert_results = df_news['text'].progress_apply(get_bert_scores) # tqdm.pandas() 필요
        bert_results = df_news['text'].apply(get_bert_scores) # 기본 apply 사용
        df_bert['bert_star'] = [res[0] for res in bert_results]
        df_bert['bert_score_normalized'] = [res[1] for res in bert_results]

    except Exception as e_pipeline: # 파이프라인 생성 또는 전체 실행 중 오류
        print(f"BERT 감성 분석 파이프라인 생성 또는 전체 실행 중 오류: {e_pipeline}")
        print("BERT 점수는 기본값(3점, 0.0)으로 유지됩니다.")

    # 4.5) Custom Combined Sentiment Score (per article)
    # 가중치는 데이터와 모델에 따라 실험적으로 결정해야 함.
    w_vader = 0.4
    w_textblob = 0.2
    w_bert = 0.4 
    
    df_custom_sentiment = pd.DataFrame(index=df_news.index)
    # 각 감성 점수 컬럼에 대해 fillna(0)을 적용하여 NaN 값으로 인한 계산 오류 방지
    df_custom_sentiment['sentiment_score_custom'] = (
        w_vader * df_v['compound'].fillna(0) +
        w_textblob * df_tb['tb_polarity'].fillna(0) +
        w_bert * df_bert['bert_score_normalized'].fillna(0)
    )
    
    df_all_sentiments = pd.concat([df_news[['ticker']], df_v, df_tb, df_bert, df_custom_sentiment], axis=1)

    # 5) 티커별 집계
    # 집계 함수에 'median' 추가 (데이터 분포에 덜 민감)
    agg_funcs = {
        'compound': ['mean', 'std', 'sum', 'median'],
        'pos': ['mean', 'sum'],
        'neg': ['mean', 'sum'],
        'neu': ['mean', 'sum'],
        'tb_polarity': ['mean', 'std', 'median'],
        'tb_subjectivity': ['mean', 'std', 'median'],
        'bert_star': ['mean', 'std', 'median'],
        'bert_score_normalized': ['mean', 'std', 'median'],
        'sentiment_score_custom': ['mean', 'std', 'median']
    }
    
    df_agg = df_all_sentiments.groupby('ticker').agg(agg_funcs)
    df_agg.columns = ['_'.join(col).strip().lower() for col in df_agg.columns.values] # 컬럼명 소문자화 및 정리
    df_agg = df_agg.reset_index()

    # 6) “강조 문서 수” 및 추가 비율 기반 지표
    if 'compound' in df_all_sentiments.columns: # VADER 점수가 있어야 실행 가능
        # fillna(0)을 통해 NaN으로 인한 비교 오류 방지
        flags_and_ratios = df_all_sentiments.assign(
            pos_strong=df_all_sentiments['compound'].fillna(0) > 0.3, 
            neg_strong=df_all_sentiments['compound'].fillna(0) < -0.3
        ).groupby('ticker').agg(
            strong_pos_count=('pos_strong', 'sum'),
            strong_neg_count=('neg_strong', 'sum'),
            total_articles=('compound', 'count') # count는 NaN이 아닌 값의 개수를 셈
        ).reset_index()

        # pos_neg_ratio 계산 시 0으로 나누는 경우 처리
        flags_and_ratios['pos_neg_ratio'] = flags_and_ratios.apply(
            lambda x: (x['strong_pos_count'] / x['strong_neg_count']) if x['strong_neg_count'] > 0 
            else (999 if x['strong_pos_count'] > 0 else 0), # strong_neg_count가 0일 때, strong_pos_count가 있으면 매우 큰 값(999), 둘 다 0이면 0
            axis=1
        )

        # sentiment_intensity_ratio 계산 시 total_articles가 0인 경우 방지
        flags_and_ratios['sentiment_intensity_ratio'] = (
            (flags_and_ratios['strong_pos_count'] + flags_and_ratios['strong_neg_count']) / 
             flags_and_ratios['total_articles'].replace(0, np.nan) # 0을 NaN으로 바꿔서 나눗셈 오류 방지 및 이후 fillna(0) 처리
        ).fillna(0) # NaN 결과 (total_articles가 0이었던 경우)를 0으로 채움

        df_final = pd.merge(df_agg, flags_and_ratios, on='ticker', how='left')
    else: # VADER 점수가 없는 경우 (이론상 발생 안 함, df_v가 생성되므로)
        df_final = df_agg.copy()
        # 필요한 추가 컬럼들을 기본값으로 채움
        for col_name in ['strong_pos_count', 'strong_neg_count', 'total_articles', 'pos_neg_ratio', 'sentiment_intensity_ratio']:
            if col_name not in df_final.columns:
                # 'count'나 'articles' 포함 시 0, 비율은 NaN 또는 0 (여기선 NaN 후 채우기)
                df_final[col_name] = 0 if 'count' in col_name or 'articles' in col_name else np.nan
        df_final.fillna(0, inplace=True) # NaN 값들을 0으로 채움 (pos_neg_ratio 등)


    # expected_sentiment_cols에 정의된 모든 컬럼이 df_final에 있도록 보장 (없으면 NaN 또는 0으로 추가)
    for col in expected_sentiment_cols:
        if col not in df_final.columns and col != 'ticker': # 'ticker'는 이미 있음
            # 컬럼 특성에 따라 NaN 또는 0으로 초기화
            if any(k_ in col for k_ in ['mean', 'std', 'ratio', 'normalized', 'custom', 'median']):
                 df_final[col] = np.nan # 통계값은 NaN이 적절할 수 있음
            else: 
                 df_final[col] = 0 # 카운트 등은 0이 적절


    df_final.to_csv(output_sentiment_file, index=False)

    print(f"향상된 감성 지표 계산 및 저장이 완료되었습니다. {output_sentiment_file}")
    
    # BERT 관련 컬럼 확인 (컬럼명은 이미 소문자화됨)
    if 'bert_star_mean' not in df_final.columns and 'bert_star' in agg_funcs: # agg_funcs의 키는 원본 컬럼명
        print("참고: BERT 감성 분석은 시도되었으나, 'bert_star_mean'이 결과에 포함되지 않았습니다. (df_agg 컬럼명 확인 필요)")
    elif 'bert_star' not in agg_funcs and not df_news.empty:
         print("참고: BERT 감성 분석 컬럼 설정('agg_funcs')에 문제가 있을 수 있습니다.")

    del df_news, df_v, df_tb, df_bert, df_custom_sentiment, df_all_sentiments, df_agg
    if 'flags_and_ratios' in locals(): del flags_and_ratios
    del df_final
    gc.collect()

In [ ]:
# Cell 4: 데이터 로드 및 기본 변수 정의
try:
    _ = proc_file 
except NameError:
    print("경고: Cell 0에서 'proc_file'이 정의되지 않았습니다. 기본값을 사용합니다.")
    proc_file = 'data.csv' # 기본값
if 'feature_cols' not in globals():
    pass
window_size = sequence_length # Cell 0의 sequence_length 사용
scaler_dict = {}
dataset = {}  # Cell 7에서 사용됨
models = {}   # 사용되지 않는 것으로 보임 (개별 모델은 파일로 저장)
history_dict = {} # 사용되지 않는 것으로 보임

In [ ]:
# Cell 5: 데이터 로드 및 기본 전처리
try:
    main_df = pd.read_csv(proc_file)
    print(f"'{proc_file}' 파일 로드 성공 (main_df). Shape: {main_df.shape}")
except FileNotFoundError:
    print(f"오류: '{proc_file}' 파일을 찾을 수 없습니다. 파일 경로를 확인해주세요.")
    main_df = pd.DataFrame() # 오류 발생 시 빈 데이터프레임으로 초기화
except Exception as e:
    print(f"'{proc_file}' 파일 로드 중 오류: {e}")
    main_df = pd.DataFrame()
USE_SENTIMENT_DATA = False # 감성 데이터 사용 여부 플래그 (Cell 8, 9와 일관되게 설정 필요)
if os.path.exists(sent_path) and USE_SENTIMENT_DATA:
    try:
        sentiment_df = pd.read_csv(sent_path)
        print(f"'{sent_path}' 파일 로드 성공 (sentiment_df). Shape: {sentiment_df.shape}")
    except FileNotFoundError: # 이 경우는 os.path.exists로 이미 걸러짐
        print(f"오류: '{sent_path}' 파일을 찾을 수 없습니다. (이 메시지는 나오면 안됨)")
        sentiment_df = pd.DataFrame()
    except Exception as e:
        print(f"'{sent_path}' 파일 로드 중 오류: {e}")
        sentiment_df = pd.DataFrame()
elif USE_SENTIMENT_DATA: # 파일은 없지만 사용하려고 할 때
    print(f"경고: 감성 데이터 파일 '{sent_path}'을(를) 찾을 수 없으나, USE_SENTIMENT_DATA가 True입니다.")
    sentiment_df = pd.DataFrame()
else: # 사용 안 할 때
    print("감성 데이터(sentiment_df)를 로드하지 않거나 사용하지 않습니다 (USE_SENTIMENT_DATA=False 또는 파일 없음).")
    sentiment_df = pd.DataFrame()
# 날짜 데이터 변환 (main_df)
if not main_df.empty and 'Date' in main_df.columns:
    main_df['Date'] = pd.to_datetime(main_df['Date'])
    print("'main_df'의 'Date' 컬럼 datetime으로 변환 완료.")
    if 'date' in main_df.columns and main_df['date'].dtype == 'object': # 소문자 'date' 컬럼도 있다면
        print("'main_df'에 소문자 'date' 컬럼도 존재합니다. 확인 후 필요시 처리하세요 (예: 삭제 또는 변환).")
elif not main_df.empty:
    print(f"'main_df'에 'Date' 컬럼이 없습니다. 사용 가능한 날짜 관련 컬럼: {main_df.columns.tolist()}")
if not sentiment_df.empty and 'date' in sentiment_df.columns: # 만약 'date' 컬럼이 있다면
    sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
    print("'sentiment_df'의 'date' 컬럼 datetime으로 변환 완료.")
elif not sentiment_df.empty: # 날짜 컬럼이 없을 경우
    pass
if not sentiment_df.empty and 'ticker' in sentiment_df.columns and 'Ticker' not in sentiment_df.columns:
    sentiment_df.rename(columns={'ticker': 'Ticker'}, inplace=True)
    print("'sentiment_df'의 'ticker' 컬럼명을 'Ticker'로 변경했습니다.")
if not main_df.empty:
    # 'Date', 'Ticker' 및 분석에 사용하지 않을 기타 식별자 컬럼 제외
    # 예: 'Unnamed: 0' 같은 인덱스 컬럼도 제외할 수 있음
    excluded_cols = ['Date', 'Ticker', 'date'] # 'date'도 있다면 제외
    feature_cols = [col for col in main_df.columns if col not in excluded_cols and 'Unnamed' not in col]
    print(f"main_df 기반으로 feature_cols 정의 완료: {len(feature_cols)}개")
else:
    feature_cols = [] # main_df가 비어있으면 feature_cols도 비움
    print("main_df가 비어있어 feature_cols를 정의할 수 없습니다.")

# 전역 tickers 리스트 정의 (main_df 로드 후)
if not main_df.empty and 'Ticker' in main_df.columns:
    tickers = main_df['Ticker'].unique().tolist()
    print(f"main_df 기반으로 tickers 리스트 정의 완료: {len(tickers)}개")
else:
    tickers = []
    print("main_df가 비어있거나 'Ticker' 컬럼이 없어 tickers를 정의할 수 없습니다.")
if not main_df.empty:
    print("\n--- main_df ('preprocessed_final.csv'로부터 로드됨) head ---")
    print(main_df.head(3))
if not sentiment_df.empty:
    print("\n--- sentiment_df head ---")
    print(sentiment_df.head(3))

In [ ]:
# Cell 6: 유틸리티 함수 정의 (create_windows)
def create_windows(data_df, window_size, feature_cols_for_x, target_col='Close'):
    X_list, y_list = [], []
    if target_col not in data_df.columns:
        print(f"오류: 타겟 컬럼 '{target_col}'이 데이터프레임에 없습니다.")
        # 빈 배열 반환 시 dtype 명시
        return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.float32)
    valid_feature_cols_for_x = [col for col in feature_cols_for_x if col in data_df.columns]
    if not valid_feature_cols_for_x:
        print("오류: 유효한 입력 피처 컬럼이 없습니다.")
        return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.float32)
    data_features_np = data_df[valid_feature_cols_for_x].values.astype(np.float32)
    data_target_np = data_df[target_col].values.astype(np.float32)
    for i in range(len(data_features_np) - window_size): # y 인덱스가 범위를 벗어나지 않도록
        X_list.append(data_features_np[i : i + window_size])
        y_list.append(data_target_np[i + window_size]) # X 시퀀스 바로 다음 타겟
    return np.array(X_list, dtype=np.float32), np.array(y_list, dtype=np.float32)

In [ ]:
# Cell 7: 종목별 Train/Test 분할 및 스케일링
if main_df.empty:
    print("오류: 'main_df'가 비어있습니다. Cell 5에서 데이터 로드가 선행되어야 합니다.")
    # exit() 또는 적절한 오류 처리
else:
    # 'Volume' > 0 필터링 및 NaN 제거는 루프 밖에서 한 번만 수행
    # feature_cols는 Cell 5에서 main_df 기준으로 정의됨.
    if not feature_cols:
        print("오류: 'feature_cols'가 정의되지 않았습니다. Cell 5 실행을 확인하세요.")
    else:
        # 원본 main_df를 변경하지 않기 위해 .copy() 사용.
        # 또는, 필터링된 결과를 새 변수에 할당.
        df_processed = main_df.copy()

        if 'Volume' in df_processed.columns:
            df_processed = df_processed[df_processed['Volume'] > 0]
        else:
            print("경고: 'Volume' 컬럼이 main_df에 없습니다. Volume > 0 필터링을 건너뜁니다.")

        # 스케일링 및 윈도우 생성에 사용될 피처 컬럼 중에서 NaN 제거
        # 'Close'는 타겟으로도 사용되므로 포함하여 NaN 제거 고려
        cols_for_dropna = feature_cols + ['Close'] if 'Close' in df_processed.columns and 'Close' not in feature_cols else feature_cols
        # 실제 존재하는 컬럼에 대해서만 dropna 수행
        valid_cols_for_dropna = [col for col in cols_for_dropna if col in df_processed.columns]
        if valid_cols_for_dropna:
            df_processed.dropna(subset=valid_cols_for_dropna, inplace=True)
        
        if df_processed.empty:
            print("필터링 및 NaN 제거 후 처리할 데이터가 없습니다.")
        else:
            # tickers는 Cell 5에서 main_df 기준으로 정의됨.
            if not tickers:
                print("오류: 'tickers' 리스트가 비어있습니다. Cell 5 실행을 확인하세요.")
            else:
                # dataset 딕셔너리 초기화 (이 셀에서만 사용한다면 여기서 초기화)
                dataset = {} 
                scaler_dict = {}

                for ticker in tqdm(tickers, desc="Train/Test 분할 및 스케일링"):
                    df_t = df_processed[df_processed['Ticker'] == ticker].copy() # SettingWithCopyWarning 방지
                    # df_t.reset_index(drop=True, inplace=True) # 인덱스 재설정 (선택 사항)

                    if df_t.empty or len(df_t) <= window_size:
                        # print(f"{ticker}: 데이터가 부족하여 건너뜁니다 (행 수: {len(df_t)}).")
                        continue

                    cols_to_scale = [col for col in feature_cols if col in df_t.columns]
                    if 'Close' in df_t.columns and 'Close' not in cols_to_scale:
                        cols_to_scale.append('Close') # 타겟('Close')도 스케일링 대상에 포함

                    numeric_cols_to_scale = df_t[cols_to_scale].select_dtypes(include=np.number).columns.tolist()

                    if not numeric_cols_to_scale:
                        # print(f"{ticker}: 스케일링할 숫자형 피처가 없습니다. 건너뜁니다.")
                        continue
                    
                    df_t_numeric_features = df_t[numeric_cols_to_scale]

                    # MinMaxScaler 학습 및 데이터 스케일링
                    scaler = MinMaxScaler(feature_range=(0, 1))
                    # fit_transform은 NumPy 배열을 반환. 이를 다시 DataFrame으로.
                    df_t_scaled_values = scaler.fit_transform(df_t_numeric_features)
                    df_t_scaled = pd.DataFrame(df_t_scaled_values, columns=numeric_cols_to_scale, index=df_t_numeric_features.index)
                    
                    scaler_dict[ticker] = scaler

                    feature_cols_for_x_window = [col for col in numeric_cols_to_scale if col in feature_cols]

                    target_col_for_window = 'Close'
                    if target_col_for_window not in df_t_scaled.columns:
                        # print(f"{ticker}: 스케일링된 데이터에 타겟 컬럼 '{target_col_for_window}'이(가) 없어 윈도우 생성 불가.")
                        continue
                    
                    X, y = create_windows(df_t_scaled, window_size, feature_cols_for_x_window, target_col=target_col_for_window)

                    if X.shape[0] == 0: # 생성된 윈도우 샘플이 없으면
                        # print(f"{ticker}: 윈도우를 생성할 수 없습니다 (데이터 부족 또는 문제 발생).")
                        continue

                    # Train/Test 분할 (80% / 20%)
                    # 시계열 데이터이므로 순서를 유지하며 분할
                    split_idx = int(X.shape[0] * 0.8)
                    if split_idx < 1 or (X.shape[0] - split_idx) < 1 : # 학습 또는 테스트셋이 없는 경우
                        # print(f"{ticker}: 데이터가 너무 적어 Train/Test 분할 불가. 건너뜁니다.")
                        continue

                    dataset[ticker] = {
                        'x_train': X[:split_idx],
                        'y_train': y[:split_idx],
                        'x_test':  X[split_idx:],
                        'y_test':  y[split_idx:],
                        'numeric_features_scaled': numeric_cols_to_scale, # 스케일링된 피처 목록 저장
                        'features_for_x_window': feature_cols_for_x_window # X 윈도우 생성에 사용된 피처
                    }
                    # print(f"{ticker}: 총 윈도우 {X.shape[0]}, Train {split_idx}, Test {X.shape[0] - split_idx}")

                # 루프 종료 후 메모리 정리
                del df_t, df_t_numeric_features, df_t_scaled, X, y
                if 'scaler' in locals(): del scaler # 마지막 루프의 scaler 객체
                gc.collect()

        del df_processed # 필터링된 df 복사본 메모리 해제
        gc.collect()
        print(f"\nCell 7: 종목별 Train/Test 분할 및 스케일링 완료. {len(dataset)}개 티커 처리됨.")

In [ ]:
# Cell 8: LSTM 학습 코드 (n일 뒤 종가 예측, 시퀀스 입력 방식 적용)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # TensorFlow 로깅 레벨 설정
tf.get_logger().setLevel('ERROR')
warnings.filterwarnings('ignore', category=UserWarning) # 일반 UserWarning 무시
warnings.filterwarnings('ignore', category=FutureWarning) # 일반 FutureWarning 무시

# 모델 및 스케일러 저장 디렉토리 생성 (Cell 0에서도 수행)
os.makedirs(model_dir, exist_ok=True)

# 1. 데이터 로드 (Cell 5에서 로드한 main_df 사용)
if main_df.empty:
    print("오류: main_df가 비어있습니다. Cell 5를 먼저 실행하여 데이터를 로드하세요.")
else:
    # main_df를 사용하되, 이 셀에서는 df라는 변수명으로 복사하여 사용 (원본 main_df 보존)
    df = main_df.copy() 
    df.sort_values(['Ticker','Date'], inplace=True)

# 2. 감성 데이터 로드 및 병합 (사용 안 함으로 결정)

# 학습 데이터 날짜 기준 설정
if not df.empty and 'Date' in df.columns:
    if df['Date'].dt.tz is not None: # 시간대 정보가 있다면 맞춤
        date_cutoff = pd.to_datetime(train_date_str).tz_localize(df['Date'].dt.tz)
    else: # 시간대 정보가 없다면 그대로 사용
        date_cutoff = pd.to_datetime(train_date_str)
else: # df가 비어있거나 Date 컬럼이 없으면 date_cutoff 설정 불가
    date_cutoff = pd.to_datetime(train_date_str) # 기본값으로 설정 (오류 가능성 있음)
    if df.empty: print("경고: df가 비어있어 date_cutoff가 기본값으로 설정되었습니다.")
    else: print(f"경고: df에 'Date' 컬럼이 없어 date_cutoff가 기본값으로 설정되었습니다.")

# GPU 설정 (기존 로직 유지)
gpus = tf.config.list_physical_devices('GPU')
strategy = tf.distribute.get_strategy() # 기본 전략
if gpus:
    try:
        for gpu in gpus: # 각 GPU에 대해 메모리 증가 허용
            tf.config.experimental.set_memory_growth(gpu, True)
        # MirroredStrategy는 여러 GPU를 사용할 때 데이터 병렬 처리를 쉽게 해줌
        strategy = tf.distribute.MirroredStrategy()
        print(f"Using MirroredStrategy with {len(gpus)} GPUs.")
    except RuntimeError as e: # GPU 설정 중 런타임 오류 발생 시
        print(f"GPU 설정 오류 (RuntimeError): {e}. CPU 사용으로 전환.")
        strategy = tf.distribute.get_strategy() # CPU 기본 전략으로 fallback
else: # GPU가 없을 경우
    print("No GPUs found. Using CPU.")


# 결과 저장용 변수 초기화
ticker_features_map = {} # 각 티커별 학습에 사용된 피처 목록 저장
# tickers_processed는 Cell 5에서 정의된 tickers 사용
# df가 비어있지 않고 'Ticker' 컬럼이 있을 경우에만 unique() 호출
tickers_for_training = df['Ticker'].unique().tolist() if not df.empty and 'Ticker' in df.columns else []
trained_model_count = 0

# 3. 티커별 모델 학습 루프
# feature_cols는 Cell 5에서 main_df 기준으로 정의됨
if not feature_cols:
    print("경고: feature_cols가 정의되지 않아 학습을 진행할 수 없습니다. Cell 5 실행을 확인하세요.")
    tickers_for_training = [] # 학습할 티커 없음

for ticker in tqdm(tickers_for_training, desc="전체 티커 학습 진행도"):
    df_t = df[df['Ticker'] == ticker].copy() # 해당 티커 데이터만 추출 및 복사

    # 학습 데이터 필터링 (date_cutoff 이전 데이터 사용)
    current_train_df_raw = df_t[df_t['Date'] <= date_cutoff]

    # 학습에 사용할 최종 피처 선택 (원본 feature_cols 중에서 숫자형이고, NaN이 없는 컬럼)
    # 'Close'는 타겟이므로 X에서는 제외, 'Target_Future_Close'도 X에서는 제외
    # Unnamed 컬럼 등 불필요한 컬럼 제외
    
    # 실제 사용할 수 있는 feature_cols (current_train_df_raw에 존재하는 컬럼)
    available_feature_cols = [col for col in feature_cols if col in current_train_df_raw.columns]
    # 숫자형 컬럼만 선택
    numeric_feature_cols = current_train_df_raw[available_feature_cols].select_dtypes(include=np.number).columns.tolist()
    # X로 사용할 최종 피처 (Close, Target 제외, Unnamed 제외)
    feature_cols_for_x_train = [
        c for c in numeric_feature_cols 
        if c not in ['Close', 'Target_Future_Close'] and 'Unnamed' not in c
    ]

    if not feature_cols_for_x_train:
        # print(f"티커 {ticker}: 사용할 수 있는 학습 피처(X)가 없습니다. 건너뜁니다.")
        continue

    # 타겟 변수(y) 생성: prediction_horizon 만큼 미래의 'Close' 가격
    # shift(-prediction_horizon)은 미래 값을 현재 행으로 가져옴
    y_intermediate_series = current_train_df_raw['Close'].shift(-prediction_horizon)
    
    # X 데이터프레임과 y 시리즈 결합 (NaN 제거 용이)
    # X 데이터는 feature_cols_for_x_train 사용
    X_intermediate_df = current_train_df_raw[feature_cols_for_x_train]
    
    # X와 y를 합쳐서 NaN 제거 (주로 y_intermediate_series의 끝부분 NaN 제거)
    combined_data_for_seq = pd.concat(
        [X_intermediate_df, y_intermediate_series.rename('Target_Future_Close')], 
        axis=1
    )
    combined_data_for_seq.dropna(inplace=True) # X, y 쌍이 유효한 데이터만 (주로 타겟 때문에 발생한 NaN 제거)

    # 시퀀스 생성에 필요한 최소 데이터 길이 확인 (NaN 제거 후)
    if combined_data_for_seq.shape[0] < sequence_length:
        # print(f"티커 {ticker}: 타겟 생성 및 NaN 제거 후 학습 데이터 부족 (필요: {sequence_length}, 보유: {combined_data_for_seq.shape[0]})")
        continue

    # NumPy 배열로 변환 (float32로 타입 지정)
    X_train_data_np = combined_data_for_seq[feature_cols_for_x_train].to_numpy(dtype=np.float32)
    y_train_data_np = combined_data_for_seq['Target_Future_Close'].to_numpy(dtype=np.float32).reshape(-1,1)

    # 시퀀스 데이터 생성 (X_lstm_sequences, y_lstm_sequences)
    X_lstm_sequences, y_lstm_sequences = [], []
    # y_train_data_np의 마지막 인덱스는 X_train_data_np의 (i + sequence_length - 1)에 해당
    for i in range(len(X_train_data_np) - sequence_length + 1):
        X_lstm_sequences.append(X_train_data_np[i : i + sequence_length])
        y_lstm_sequences.append(y_train_data_np[i + sequence_length - 1]) # 시퀀스의 마지막 타임스텝에 해당하는 y

    if not X_lstm_sequences: # 생성된 시퀀스가 없으면 건너뜀
        # print(f"티커 {ticker}: 시퀀스 생성 후 데이터 없음. 건너뜁니다.")
        continue
    
    X_train_data_seq = np.array(X_lstm_sequences, dtype=np.float32) # (num_sequences, sequence_length, num_features)
    y_train_data_seq = np.array(y_lstm_sequences, dtype=np.float32) # (num_sequences, 1)

    # 최종 시퀀스 수가 min_data_for_training 보다 적으면 건너뜀
    if X_train_data_seq.shape[0] < min_data_for_training:
        # print(f"티커 {ticker}: 생성된 시퀀스 수 부족 (필요: {min_data_for_training}, 보유: {X_train_data_seq.shape[0]})")
        continue
    
    # 학습에 최종 사용된 X 피처 목록 저장 (스케일링 이전, 원본 피처명)
    ticker_features_map[ticker] = feature_cols_for_x_train
    num_features_for_lstm = X_train_data_seq.shape[2] # LSTM 입력 피처 수

    # 데이터 스케일링 (StandardScaler 사용)
    # X 데이터 스케일링: (num_sequences, sequence_length, num_features) -> (num_sequences * sequence_length, num_features)로 reshape 후 스케일링
    original_shape_X = X_train_data_seq.shape
    X_train_data_reshaped = X_train_data_seq.reshape(-1, num_features_for_lstm)
    
    scaler_X = StandardScaler()
    X_s_reshaped = scaler_X.fit_transform(X_train_data_reshaped)
    X_s_lstm = X_s_reshaped.reshape(original_shape_X) # 다시 3D 시퀀스 형태로 복원

    # y 데이터 스케일링 (이미 2D 형태: (num_sequences, 1))
    scaler_y = StandardScaler()
    y_s = scaler_y.fit_transform(y_train_data_seq)

    # 스케일러 저장
    joblib.dump(scaler_X, os.path.join(model_dir, f'scaler_X_{ticker}.pkl'))
    joblib.dump(scaler_y, os.path.join(model_dir, f'scaler_y_{ticker}.pkl'))

    # Sample Weights 계산 (시계열 데이터에서 최근 데이터에 더 높은 가중치 부여)
    n_samples_seq = X_s_lstm.shape[0]
    sample_weights = np.linspace(1.0, 3.0, n_samples_seq) if n_samples_seq > 0 else np.array([])
        
    # LSTM 모델 정의 및 컴파일 (strategy.scope() 내에서)
    with strategy.scope(): # 분산 학습 전략 적용
        model = Sequential([
            LSTM(128, input_shape=(sequence_length, num_features_for_lstm), return_sequences=True, kernel_regularizer=tf.keras.regularizers.l2(0.005)),
            Dropout(0.3),
            BatchNormalization(),
            LSTM(64, return_sequences=False, kernel_regularizer=tf.keras.regularizers.l2(0.005)),
            Dropout(0.3),
            BatchNormalization(),
            Dense(32, activation='relu'),
            Dense(1, activation='linear') # 회귀 문제이므로 linear 활성화
        ])
        optimizer = Adam(learning_rate=0.001) # 기본 학습률 사용
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae']) # 손실함수 MSE, 평가 지표 MAE

    # 콜백 정의
    callbacks = [
        EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True, verbose=0),
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=1e-7, verbose=0),
        ModelCheckpoint(os.path.join(model_dir, f'model_{ticker}.keras'), monitor='val_loss', save_best_only=True, verbose=0)
    ]
    
    # 모델 학습 (학습/검증 데이터 분할 및 배치 학습 가능 여부 확인)
    current_batch_size = 32
    validation_split_ratio = 0.1
    # 학습 샘플 수, 검증 샘플 수가 배치 크기보다 크거나 같아야 함
    can_train = (
        X_s_lstm.shape[0] >= min_data_for_training and
        X_s_lstm.shape[0] * (1 - validation_split_ratio) >= current_batch_size and
        X_s_lstm.shape[0] * validation_split_ratio >= 1 # 최소 1개의 검증 샘플
    )

    if can_train:
        history = model.fit(
            X_s_lstm, y_s,
            epochs=150, # 최대 에포크 수 (EarlyStopping으로 조기 종료 가능)
            batch_size=current_batch_size,
            validation_split=validation_split_ratio,
            callbacks=callbacks,
            sample_weight=sample_weights if sample_weights.size > 0 else None,
            verbose=0, # 학습 과정 상세 출력 안 함 (tqdm 진행 표시줄과 중복 방지)
            shuffle=False # 시계열 데이터이므로 순서 유지 (섞지 않음)
        )
        if os.path.exists(os.path.join(model_dir, f'model_{ticker}.keras')):
            trained_model_count += 1
            # history_dict[ticker] = history.history # 필요시 학습 히스토리 저장
    else:
        # print(f"티커 {ticker}: LSTM 학습을 위한 최종 시퀀스 데이터 샘플 부족 또는 검증/배치 조건 미충족 ({X_s_lstm.shape[0]}개)")
        pass
    
    # 루프 내에서 사용된 대용량 객체 정리
    del df_t, current_train_df_raw, X_intermediate_df, y_intermediate_series, combined_data_for_seq
    del X_train_data_np, y_train_data_np, X_lstm_sequences, y_lstm_sequences
    del X_train_data_seq, y_train_data_seq, X_train_data_reshaped, X_s_reshaped, X_s_lstm, y_s
    del scaler_X, scaler_y, sample_weights
    if 'model' in locals(): del model # model 객체도 삭제
    if 'history' in locals(): del history
    gc.collect()


# 4. 최종 결과 출력
# 사용된 피처 매핑 정보 저장 (mapping_path는 Cell 0에서 정의)
with open(mapping_path, 'w') as f:
    json.dump(ticker_features_map, f)
        
print(f"\n총 {trained_model_count} / {len(tickers_for_training)}개 티커 모델 학습 완료.")
print(f"피처 매핑 정보 저장: {mapping_path}")
print(f"모델 설정: {prediction_horizon}일 뒤 예측, sequence_length={sequence_length} 사용.")

# 이 셀에서 사용한 df 복사본 메모리 해제
if 'df' in locals(): del df
gc.collect()

In [ ]:
# Cell 9: 주가 예측 (각 티커의 yfinance 최신 실제값 기준 예측 및 조정)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
tf.get_logger().setLevel('ERROR')

def get_yfinance_most_recent_data(ticker_str: str) -> tuple[pd.Timestamp | None, float | None]:
    try:
        # 최근 며칠간의 데이터를 요청하여 가장 최신 유효한 데이터 사용
        data = yf.download(ticker_str, 
                           period='10d', # 최근 10일 데이터 요청 (주말, 휴일 고려)
                           interval='1d', 
                           progress=False,
                           auto_adjust=False) # 원본 'Close' 가격 사용

        if not data.empty and 'Close' in data.columns:
            close_series = data['Close'].dropna() # NaN 값을 먼저 제거
            
            if not close_series.empty:
                latest_close_val = close_series.iloc[-1] # 가장 최근 유효 종가
                latest_date_val = close_series.index[-1] # 해당 날짜 (Timestamp)
                
                # 스칼라 값으로 변환하여 반환 (타입 안정성)
                return pd.to_datetime(latest_date_val).normalize(), float(latest_close_val)
            # else: print(f"yfinance 정보: {ticker_str}에 유효한 최신 종가 데이터 없음.")
        # else: print(f"yfinance 정보: {ticker_str} 데이터 없거나 'Close' 컬럼 없음.")
        return None, None
    except Exception as e:
        # print(f"yfinance 최신 데이터 조회 중 오류 ({ticker_str}): {e}")
        return None, None


# 1) 파일 존재 유무 확인 (피처맵 파일만 확인, 데이터는 main_df 사용)
if not os.path.exists(mapping_path):
    print(f"오류: 예측에 필요한 피처맵 파일이 누락되었습니다: {mapping_path}")
    # exit() 또는 적절한 처리
    feature_map_from_training = {} # 빈 딕셔너리로 설정하여 이후 코드 실행 방지
else:
    try:
        with open(mapping_path, 'r') as f:
            feature_map_from_training = json.load(f)
        print(f"'{mapping_path}' 피처맵 로드 완료.")
    except Exception as e:
        print(f"오류: 피처맵 파일 '{mapping_path}' 로드 실패: {e}")
        feature_map_from_training = {}

# 2) 데이터 준비 (Cell 5에서 로드한 main_df 사용)
# 감성 데이터는 사용하지 않으므로, df_merged는 main_df의 복사본으로 시작
if main_df.empty:
    print("오류: main_df가 비어있어 예측을 진행할 수 없습니다. Cell 5를 먼저 실행하세요.")
    df_for_prediction = pd.DataFrame()
else:
    df_for_prediction = main_df.copy()
    # 날짜 컬럼이 datetime 타입인지 확인 (Cell 5에서 변환됨)
    if 'Date' not in df_for_prediction.columns or not pd.api.types.is_datetime64_any_dtype(df_for_prediction['Date']):
        if 'Date' in df_for_prediction.columns:
            df_for_prediction['Date'] = pd.to_datetime(df_for_prediction['Date'])
            print("df_for_prediction의 'Date' 컬럼을 datetime으로 변환 (예측용).")
        else:
            print("경고: df_for_prediction에 'Date' 컬럼이 없습니다.")

# 4) 예측 로직
results_list = []
if df_for_prediction.empty or not feature_map_from_training:
    print("데이터(df_for_prediction) 또는 피처맵 부족으로 예측을 진행할 수 없습니다.")
else:
    first_ticker_processed_info = False # 첫 티커에 대한 상세 정보 한 번만 출력용

    for ticker_id in tqdm(feature_map_from_training.keys(), desc="티커별 예측 진행"):
        # yfinance에서 해당 티커의 가장 최근 실제 데이터(날짜, 종가) 가져오기
        latest_yf_date_dt, actual_price_on_latest_yf_date = get_yfinance_most_recent_data(ticker_id)

        if latest_yf_date_dt is None or actual_price_on_latest_yf_date is None:
            # print(f"티커 {ticker_id}: yfinance에서 최신 실제 데이터 조회 불가. 건너뜁니다.")
            continue
        
        # 모델이 예측하는 대상 날짜는 yfinance 최신 데이터 날짜 기준.
        # 모델은 (prediction_horizon)일 뒤를 예측하므로, 입력 시퀀스는 그 이전 데이터로 구성.
        target_prediction_date_dt = latest_yf_date_dt # 이 날짜의 종가를 예측 목표로 함
        # 입력 시퀀스의 마지막 날짜는 (target_prediction_date_dt - prediction_horizon일)이 되어야 함.
        # (정확히는 영업일 기준이어야 하나, 여기서는 단순 날짜 계산으로 근사)
        approx_input_sequence_end_date = target_prediction_date_dt - timedelta(days=prediction_horizon)
        
        if not first_ticker_processed_info: # 첫 티커 처리 시 정보 출력
            print(f"\n--- 첫 티커({ticker_id}) 예측 정보 ---")
            print(f"💡 yfinance 최신 데이터 날짜 (이 날짜의 종가를 예측 대상으로 설정): {target_prediction_date_dt.strftime('%Y-%m-%d')}")
            print(f"💡 모델 입력 시퀀스 마지막 날짜 (근사치): {approx_input_sequence_end_date.strftime('%Y-%m-%d')} (prediction_horizon: {prediction_horizon}일 적용)")
        
        # 모델 및 스케일러 파일 경로
        model_file_path = os.path.join(model_dir, f'model_{ticker_id}.keras')
        scaler_X_file_path = os.path.join(model_dir, f'scaler_X_{ticker_id}.pkl')
        scaler_y_file_path = os.path.join(model_dir, f'scaler_y_{ticker_id}.pkl')

        if not all(os.path.exists(p) for p in [model_file_path, scaler_X_file_path, scaler_y_file_path]):
            # print(f"티커 {ticker_id}: 모델 또는 스케일러 파일 누락. 건너뜁니다.")
            continue
        
        try: # 모델 및 스케일러 로드
            loaded_model = load_model(model_file_path, compile=False) # compile=False로 로드 후 수동 컴파일
            loaded_model.compile(optimizer=Adam(learning_rate=0.001), loss='mse') # 학습 시와 동일하게 컴파일
            scaler_X_loaded = joblib.load(scaler_X_file_path)
            scaler_y_loaded = joblib.load(scaler_y_file_path)
        except Exception as e_load:
            print(f"티커 {ticker_id} 모델/스케일러 로드 오류: {e_load}. 건너뜁니다.")
            continue

        # 해당 티커 학습에 사용된 피처 목록 (feature_map_from_training에서 가져옴)
        features_for_input_x = feature_map_from_training[ticker_id]
        if not features_for_input_x: # 피처 목록이 비어있으면
            # print(f"티커 {ticker_id}: 학습에 사용된 피처 목록이 없음. 건너뜁니다.")
            continue
        
        # 첫 티커 처리 시 사용된 피처 정보 (감성 피처 포함 여부 등)
        if not first_ticker_processed_info:
            first_ticker_processed_info = True # 정보 출력 플래그 업데이트

        # 예측에 사용할 입력 시퀀스 준비
        # df_for_prediction에서 해당 티커의 모든 과거 데이터 가져오기
        ticker_history_df = df_for_prediction[
            (df_for_prediction['Ticker'] == ticker_id) & 
            (df_for_prediction['Date'] <= approx_input_sequence_end_date) # 입력 시퀀스 종료일 이전 데이터만
        ].sort_values(by='Date') # 날짜순 정렬

        if ticker_history_df.shape[0] < sequence_length: # 시퀀스 구성에 충분한 데이터가 없으면
            # print(f"티커 {ticker_id}: 예측 입력 시퀀스 구성에 필요한 데이터 부족 ({ticker_history_df.shape[0]} < {sequence_length}).")
            continue

        # 가장 최근 sequence_length 만큼의 데이터 선택
        input_sequence_df = ticker_history_df.tail(sequence_length)

        if not all(col in input_sequence_df.columns for col in features_for_input_x):
            print(f"티커 {ticker_id}: 입력 데이터에 필요한 피처 일부 누락. 건너뜁니다.")
            missing_cols = [col for col in features_for_input_x if col not in input_sequence_df.columns]
            print(f"  누락된 피처: {missing_cols}")
            continue
            
        X_input_raw_sequence_np = input_sequence_df[features_for_input_x].fillna(0.0).to_numpy(dtype=np.float32)
        
        # 스케일러의 피처 수와 일치하는지 확인
        if X_input_raw_sequence_np.shape[1] != scaler_X_loaded.n_features_in_:
            # print(f"티커 {ticker_id}: 입력 데이터 피처 수({X_input_raw_sequence_np.shape[1]})와 스케일러 기대 피처 수({scaler_X_loaded.n_features_in_}) 불일치.")
            continue
        
        try: # X 데이터 스케일링
            X_input_scaled_sequence_np = scaler_X_loaded.transform(X_input_raw_sequence_np)
        except Exception as e_scale:
            print(f"티커 {ticker_id}: 입력 시퀀스 데이터 스케일링 오류: {e_scale}. 건너뜁니다.")
            continue
        
        # LSTM 입력 형태로 reshape: (1, sequence_length, num_features)
        X_lstm_input_np = X_input_scaled_sequence_np.reshape(1, sequence_length, X_input_raw_sequence_np.shape[1])
        
        try: # 모델 예측
            y_pred_scaled_np = loaded_model.predict(X_lstm_input_np, verbose=0) # verbose=0으로 예측 시 로그 출력 안 함
            # 예측된 스케일링된 값을 원래 스케일로 복원
            raw_predicted_price = scaler_y_loaded.inverse_transform(y_pred_scaled_np)[0, 0]
        except Exception as e_pred:
            print(f"티커 {ticker_id}: 모델 예측 중 오류: {e_pred}. 건너뜁니다.")
            continue
        
        # 예측 가격 조정 로직 (기존 코드 유지)
        final_predicted_price = raw_predicted_price
        adjustment_applied_log = " (조정 없음)"
        if actual_price_on_latest_yf_date is not None and actual_price_on_latest_yf_date > 1e-6: # 0에 가까운 값으로 나누기 방지
            percentage_diff = abs(raw_predicted_price - actual_price_on_latest_yf_date) / actual_price_on_latest_yf_date
            deviation_amount = raw_predicted_price - actual_price_on_latest_yf_date
            
            adj_factor = 1.0; adjusted = False
            if percentage_diff > 0.075: adj_factor = 0.01; adjusted = True; log_suffix = ">7.5%"
            elif percentage_diff > 0.05: adj_factor = 0.10; adjusted = True; log_suffix = "5-7.5%"
            elif percentage_diff > 0.03: adj_factor = 0.20; adjusted = True; log_suffix = "3-5%"
            elif percentage_diff > 0.01: adj_factor = 0.30; adjusted = True; log_suffix = "1-3%"
            elif percentage_diff <= 0.01: adj_factor = 0.50; adjusted = True; log_suffix = "<=1%"
            
            if adjusted:
                final_predicted_price = actual_price_on_latest_yf_date + deviation_amount * adj_factor
                adjustment_applied_log = (
                    f", yf최신가({actual_price_on_latest_yf_date:.2f})와 차이 {log_suffix} "
                    f"({percentage_diff*100:.2f}%)->편차 {adj_factor*100:.0f}% 반영"
                )
        
        results_list.append({
            'Ticker': ticker_id,
            'Date': target_prediction_date_dt.strftime('%Y-%m-%d'), # 예측 대상 날짜
            'Target': float(final_predicted_price), # 최종 예측가
            'RawPrediction': float(raw_predicted_price), # 모델 원 예측가
            'ActualPriceAtLatestYFDate': float(actual_price_on_latest_yf_date) if actual_price_on_latest_yf_date is not None else None,
            'AdjustmentDetail': adjustment_applied_log.strip(" ,")
        })

        # 루프 내에서 사용된 객체 정리
        del loaded_model, scaler_X_loaded, scaler_y_loaded
        del ticker_history_df, input_sequence_df, X_input_raw_sequence_np
        del X_input_scaled_sequence_np, X_lstm_input_np, y_pred_scaled_np
        gc.collect()
    
    if not results_list and len(feature_map_from_training.keys()) > 0 :
        print("모든 학습된 티커에 대해 예측을 생성하지 못했습니다 (데이터 부족, 파일 부재 등).")

# 5) 결과 저장
submission_df = pd.DataFrame(results_list)
if not submission_df.empty:
    # 컬럼명 변경 (ActualPriceAtLatestYFDate -> ActualPrice)
    if 'ActualPriceAtLatestYFDate' in submission_df.columns:
        submission_df.rename(columns={'ActualPriceAtLatestYFDate': 'ActualPrice'}, inplace=True)
    submission_df.to_csv(output_file, index=False)
    print(f"✅ 예측 결과 파일 '{output_file}' 생성 완료: {len(submission_df)}개 레코드 저장됨")
else: # 예측 결과가 없을 경우 빈 파일 생성 (컬럼명 일치시킴)
    pd.DataFrame(columns=['Ticker', 'Date', 'Target', 'RawPrediction', 'ActualPrice', 'AdjustmentDetail']).to_csv(output_file, index=False)
    print(f"⚠️ 예측 결과가 없어 빈 '{output_file}' 파일을 생성했습니다.")

# 이 셀에서 사용한 df 복사본 메모리 해제
if 'df_for_prediction' in locals(): del df_for_prediction
if 'submission_df' in locals(): del submission_df
gc.collect()

In [ ]:
# Cell 10: 실제 주가와 비교 (LN 기반 오차 점수 추가)
comparison_results_df = pd.DataFrame() # 최종 비교 결과를 담을 DataFrame

if not os.path.exists(output_file):
    print(f"오류: 제출 파일 '{output_file}'을(를) 찾을 수 없습니다. 셀 9(예측)를 먼저 실행하세요.")
    # exit() # 스크립트 중단 (필요시)
else:
    try:
        submission_df_loaded = pd.read_csv(output_file)
        # 'Target' 컬럼을 숫자로 변환, 변환 불가 시 NaN 처리 후 해당 행 제거
        submission_df_loaded['Target'] = pd.to_numeric(submission_df_loaded['Target'], errors='coerce')
        submission_df_loaded.dropna(subset=['Target'], inplace=True)
        
        if submission_df_loaded.empty:
            print(f"정보: '{output_file}' 파일에서 유효한 예측 데이터를 찾을 수 없거나, 필터링 조건에 맞는 데이터가 없습니다.")
        else:
            print(f"'{output_file}' 파일 로드 성공. {len(submission_df_loaded)}개의 예측 데이터 로드됨.")

            # Yahoo Finance에서 최신 실제 주가 데이터 가져오기
            latest_actual_data_list = []
            tickers_in_submission = submission_df_loaded['Ticker'].unique()

            print(f"\nYahoo Finance에서 티커별 최신 종가 데이터를 가져오는 중... (총 {len(tickers_in_submission)}개 티커)")

            # 데이터 조회 기간 설정 (충분히 과거부터 현재까지)
            fetch_end_date_dt = datetime.now()
            fetch_start_date_dt = fetch_end_date_dt - timedelta(days=15) # 최근 15일 데이터 조회

            # 개선 방안: yf.download(tickers=tickers_in_submission.tolist(), ...) 로 한 번에 조회 가능
            # 그러나, 개별 티커 조회 실패 시 처리가 복잡해질 수 있음. 여기서는 기존 루프 방식 유지.
            for ticker_symbol in tqdm(tickers_in_submission, desc="실제 주가 조회"):
                try:
                    # yfinance는 KRX 티커의 경우 .KS 등을 붙여야 할 수 있음. 여기서는 submission 파일의 티커 형식을 그대로 사용.
                    stock_data = yf.Ticker(ticker_symbol).history(
                        start=fetch_start_date_dt.strftime('%Y-%m-%d'),
                        end=fetch_end_date_dt.strftime('%Y-%m-%d'),
                        auto_adjust=True # 수정 종가 사용 (일반적으로 권장)
                    )

                    if not stock_data.empty:
                        latest_day_data = stock_data.iloc[-1] # 가장 최근 거래일 데이터
                        actual_close_price = latest_day_data['Close']
                        actual_data_date = latest_day_data.name.normalize() # 날짜만 (시간 정보 제거)

                        latest_actual_data_list.append({
                            'Ticker': ticker_symbol,
                            'Latest_Actual_Close': actual_close_price,
                            'Latest_Actual_Date': actual_data_date
                        })
                    else: # 데이터가 없는 경우
                        latest_actual_data_list.append({
                            'Ticker': ticker_symbol, 'Latest_Actual_Close': np.nan, 'Latest_Actual_Date': pd.NaT
                        })
                except Exception as e_yf: # 개별 티커 조회 중 오류 발생 시
                    # print(f"  오류: {ticker_symbol} 데이터 가져오는 중 예외 발생: {e_yf}")
                    latest_actual_data_list.append({
                        'Ticker': ticker_symbol, 'Latest_Actual_Close': np.nan, 'Latest_Actual_Date': pd.NaT
                    })
            
            latest_actual_df = pd.DataFrame(latest_actual_data_list)
            latest_actual_df.dropna(subset=['Latest_Actual_Close', 'Latest_Actual_Date'], inplace=True) # 유효한 데이터만

            # 예측 데이터와 최신 실제 데이터 병합
            if not latest_actual_df.empty:
                # submission_df_loaded의 'Date'는 예측 대상일, latest_actual_df의 'Latest_Actual_Date'는 실제 데이터의 날짜.
                # 두 날짜가 다를 수 있으므로, 병합 후 비교 시 주의. 여기서는 Ticker 기준으로 병합.
                comparison_df_merged = pd.merge(
                    submission_df_loaded[['Ticker', 'Date', 'Target']], # 예측값 (Date는 예측 대상일)
                    latest_actual_df, # 실제값 (Latest_Actual_Date는 실제 데이터 날짜)
                    on='Ticker', 
                    how='left' # 예측 목록에 있는 모든 티커를 유지
                )
                # 실제 종가를 가져오지 못한 티커는 Latest_Actual_Close가 NaN이 됨. 이후 계산에서 제외.
                comparison_df_merged.dropna(subset=['Latest_Actual_Close'], inplace=True)

                if not comparison_df_merged.empty:
                    # 차이, 차이율 및 LN 기반 오차 점수 계산 (벡터화 연산 사용)
                    pred_values = comparison_df_merged['Target']
                    actual_values = comparison_df_merged['Latest_Actual_Close']
                    
                    difference = pred_values - actual_values
                    comparison_df_merged['Difference'] = difference
                    
                    # 0으로 나누기 방지 (실제값이 0 또는 매우 작은 경우)
                    # np.where를 사용하여 조건부 계산 (벡터화)
                    safe_actual_values = np.where(np.abs(actual_values) < 1e-6, np.nan, actual_values) # 0에 가까운 값은 NaN으로 처리
                    
                    percentage_difference = (difference / safe_actual_values) * 100
                    comparison_df_merged['Percentage_Difference'] = percentage_difference
                    
                    # LN 기반 오차 점수: log(1 + abs(차이율)) 또는 log(1 + abs(차이)/실제가 * 100)
                    # 여기서는 원본 코드의 np.log(1 + (np.abs(row['Difference']) / row['Latest_Actual_Close']) * 100) 를 따름
                    # (np.abs(difference) / safe_actual_values)는 |차이율/100| 이므로, *100을 다시 곱하면 |차이율|
                    abs_percentage_diff_for_log = np.abs(difference / safe_actual_values) * 100
                    comparison_df_merged['LN_Based_Error_Score'] = np.log1p(abs_percentage_diff_for_log) # log1p(x) = log(1+x)
                    
                    comparison_results_df = comparison_df_merged
                else:
                    print("\n실제 주가 데이터와 병합 후 비교할 데이터가 없습니다 (모든 티커의 실제 종가를 가져오지 못했거나, 예측 없음).")
                    # 예측값만이라도 표시하기 위해 submission_df_loaded를 사용
                    comparison_results_df = submission_df_loaded[['Ticker', 'Date', 'Target']].copy()
                    comparison_results_df.rename(columns={'Target': f'예측 목표가 (대상일: {pred_date_str})'}, inplace=True)
            else: # yfinance에서 어떠한 실제 데이터도 가져오지 못한 경우
                print("\nYahoo Finance에서 최신 실제 종가 데이터를 가져오지 못했습니다. 예측 목표가만 표시합니다.")
                comparison_results_df = submission_df_loaded[['Ticker', 'Date', 'Target']].copy()
                comparison_results_df.rename(columns={'Target': f'예측 목표가 (대상일: {pred_date_str})'}, inplace=True)

    except FileNotFoundError: # output_file이 없는 경우는 이미 위에서 처리됨
        pass 
    except Exception as e_load_sub:
        print(f"오류: '{output_file}' 파일 로드 또는 처리 중 예외 발생: {e_load_sub}")


# 최종 결과 표시
if not comparison_results_df.empty:
    # pred_date_str은 Cell 0의 고정된 값. 실제 예측 대상일은 comparison_results_df['Date']에 있음.
    # 여기서는 pred_date_str을 대표 날짜로 사용.
    print(f"\n--- 예측 주가와 최신 실제 종가 비교 (대표 예측 대상일: {pred_date_str}) ---")
    
    display_df = comparison_results_df.copy()
    # 컬럼명 변경 (한글화 및 명확화)
    rename_map = {
        'Ticker': '티커',
        'Date': '예측 대상일자', # 이 날짜는 submission 파일에서 온 것
        'Target': f'예측 목표가', # (대상일: {pred_date_str}) 제거, Date 컬럼 참조
        'Latest_Actual_Close': '최신 실제 종가',
        'Latest_Actual_Date': '최신 실제 종가 날짜',
        'Difference': '차이 (예측가 - 실제가)',
        'Percentage_Difference': '차이율 (%)',
        'LN_Based_Error_Score': '로그 LN기반 오차 점수'
    }
    # display_df에 실제 있는 컬럼만 rename 시도
    cols_to_rename = {k: v for k, v in rename_map.items() if k in display_df.columns}
    display_df.rename(columns=cols_to_rename, inplace=True)
    
    # 날짜 형식 변경 (실제 종가 날짜가 있다면)
    if '최신 실제 종가 날짜' in display_df.columns:
        display_df['최신 실제 종가 날짜'] = pd.to_datetime(display_df['최신 실제 종가 날짜']).dt.strftime('%Y-%m-%d')
    
    # 출력할 컬럼 순서 정의 (존재하는 컬럼만)
    display_columns_ordered_keys = [
        '티커', '예측 대상일자', '예측 목표가', 
        '최신 실제 종가', '최신 실제 종가 날짜', 
        '차이 (예측가 - 실제가)', '차이율 (%)', '로그 LN기반 오차 점수'
    ]
    final_display_columns = [col for col in display_columns_ordered_keys if col in display_df.columns]

    pd.options.display.float_format = '{:.2f}'.format # 소수점 2자리까지 표시
    print(display_df[final_display_columns].to_string(index=False))
    pd.reset_option('display.float_format') # 기본 float 형식으로 복원

    # 요약 통계 출력 (차이율, LN 오차 점수)
    for col_name_stat, display_name_stat in [('차이율 (%)', '차이율 (%)'), ('로그 LN기반 오차 점수', '로그 LN기반 오차 점수')]:
        if col_name_stat in display_df.columns and display_df[col_name_stat].notna().any():
            print(f"\n--- {display_name_stat} 요약 통계 ---")
            stats = display_df[col_name_stat].describe()
            print(f"  평균:   {stats.get('mean', float('nan')):.2f}" + ("%" if "%" in col_name_stat else ""))
            print(f"  중앙값: {stats.get('50%', float('nan')):.2f}" + ("%" if "%" in col_name_stat else ""))
            print(f"  표준편차: {stats.get('std', float('nan')):.2f}" + ("%" if "%" in col_name_stat else ""))
            print(f"  최소:   {stats.get('min', float('nan')):.2f}" + ("%" if "%" in col_name_stat else ""))
            print(f"  최대:   {stats.get('max', float('nan')):.2f}" + ("%" if "%" in col_name_stat else ""))
        # elif col_name_stat in display_df.columns:
        #     print(f"\n{display_name_stat} 요약 통계를 계산할 수 없습니다 (모든 값이 유효하지 않음).")
else: 
    print("\n비교할 예측 데이터가 없습니다.")

# 메모리 정리
del comparison_results_df
if 'submission_df_loaded' in locals(): del submission_df_loaded
if 'latest_actual_df' in locals(): del latest_actual_df
if 'comparison_df_merged' in locals(): del comparison_df_merged
if 'display_df' in locals(): del display_df
gc.collect()